<a href="https://colab.research.google.com/github/ValentinaEmili/Ethnicity-recognition/blob/main/majority_voting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
from tqdm import tqdm
import os
from sklearn.metrics import classification_report, accuracy_score

def merged_results(path):
  cnn_pred = pd.read_csv(f"{path}/CNN_predictions.csv")
  lightGBM_pred = pd.read_csv(f"{path}/LightGBM_predictions.csv")
  alexnet_pred = pd.read_csv(f"{path}/alexnet_predictions.csv")
  svm_pred = pd.read_csv(f"{path}/svm_predictions.csv")
  y_true = pd.read_csv(f"{path}/true_values.csv", index_col=False)

  cnn_pred["Image_Name"] = cnn_pred["Image_Name"].str.replace(".jpg", "", regex=False)
  lightGBM_pred["Image_Name"] = lightGBM_pred["Image_Name"].str.replace(".jpg", "", regex=False)
  alexnet_pred["Image_Name"] = alexnet_pred["Image_Name"].str.replace(".jpg", "", regex=False)
  svm_pred["Image_Name"] = svm_pred["Image_Name"].str.replace(".npy", "", regex=False)
  y_true["Image_Name"] = y_true["Image_Name"].str.replace(".npy", "", regex=False)

  cnn_pred = cnn_pred.set_index("Image_Name").sort_index()
  lightGBM_pred = lightGBM_pred.set_index("Image_Name").sort_index()
  alexnet_pred = alexnet_pred.set_index("Image_Name").sort_index()
  svm_pred = svm_pred.set_index("Image_Name").sort_index()
  y_true = y_true.set_index("Image_Name").sort_index()

  cnn_pred.rename(columns={"Predicted_Class": "Predicted_class_cnn"}, inplace=True)
  lightGBM_pred.rename(columns={"Predicted_Label": "Predicted_class_lightGBM"}, inplace=True)
  alexnet_pred.rename(columns={"Predicted_Class": "Predicted_class_alexnet"}, inplace=True)
  svm_pred.rename(columns={"Predicted_Class": "Predicted_class_svm"}, inplace=True)

  merged = pd.concat([y_true, cnn_pred["Predicted_class_cnn"], lightGBM_pred["Predicted_class_lightGBM"], alexnet_pred["Predicted_class_alexnet"], svm_pred["Predicted_class_svm"]], axis=1)
  return merged

In [ ]:
# ethnicity

def get_predictions_ethnicity(row, feature):
  def get_prediction_eyes(row):
    counter = {0: 0, 1: 0, 2: 0, 3: 0, 4: 0}
    counter[row["Predicted_class_cnn"]] += 0.3
    counter[row["Predicted_class_lightGBM"]] += 0.1
    counter[row["Predicted_class_alexnet"]] += 0.3
    counter[row["Predicted_class_svm"]] += 0.3
    return int(max(counter, key=counter.get))

  def get_prediction_mouth(row):
    counter = {0: 0, 1: 0, 2: 0, 3: 0, 4: 0}
    counter[row["Predicted_class_cnn"]] += 0.5
    counter[row["Predicted_class_lightGBM"]] += 0.2
    counter[row["Predicted_class_alexnet"]] += 0.4
    counter[row["Predicted_class_svm"]] += 0.4
    return int(max(counter, key=counter.get))

  def get_prediction_nose(row):
    counter = {0: 0, 1: 0, 2: 0, 3: 0, 4: 0}
    counter[row["Predicted_class_cnn"]] += 0.42
    counter[row["Predicted_class_lightGBM"]] += 0.1
    counter[row["Predicted_class_alexnet"]] += 0.5
    counter[row["Predicted_class_svm"]] += 0.6
    return int(max(counter, key=counter.get))

  if feature == "eyes":
    return get_prediction_eyes(row)
  if feature == "mouth":
    return get_prediction_mouth(row)
  if feature == "nose":
    return get_prediction_nose(row)

In [ ]:
# age

def get_predictions_age(row, feature):
  def get_prediction_eyes(row):
    counter = {0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5:0, 6:0, 7:0}
    counter[row["Predicted_class_cnn"]] += 1
    counter[row["Predicted_class_lightGBM"]] += 0.3
    counter[row["Predicted_class_alexnet"]] += 0.3
    counter[row["Predicted_class_svm"]] += 1
    return int(max(counter, key=counter.get))

  def get_prediction_mouth(row):
    counter = {0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5:0, 6:0, 7:0}
    counter[row["Predicted_class_cnn"]] += 0.5
    counter[row["Predicted_class_lightGBM"]] += 0.3
    counter[row["Predicted_class_alexnet"]] += 0.3
    counter[row["Predicted_class_svm"]] += 0.4
    return int(max(counter, key=counter.get))

  def get_prediction_nose(row):
    counter = {0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5:0, 6:0, 7:0}
    counter[row["Predicted_class_cnn"]] += 0.5
    counter[row["Predicted_class_lightGBM"]] += 0.3
    counter[row["Predicted_class_alexnet"]] += 0.6
    counter[row["Predicted_class_svm"]] += 0.6
    return int(max(counter, key=counter.get))

  if feature == "eyes":
    return get_prediction_eyes(row)
  if feature == "mouth":
    return get_prediction_mouth(row)
  if feature == "nose":
    return get_prediction_nose(row)

In [ ]:
# gender

def get_predictions_gender(row, feature):
  def get_prediction_eyes(row):
    counter = {0: 0, 1: 0}
    counter[row["Predicted_class_cnn"]] += 0.3
    counter[row["Predicted_class_lightGBM"]] += 0.1
    counter[row["Predicted_class_alexnet"]] += 0.3
    counter[row["Predicted_class_svm"]] += 0.3
    return int(max(counter, key=counter.get))

  def get_prediction_mouth(row):
    counter = {0: 0, 1: 0}
    counter[row["Predicted_class_cnn"]] += 0.6
    counter[row["Predicted_class_lightGBM"]] += 0.2
    counter[row["Predicted_class_alexnet"]] += 0.6
    counter[row["Predicted_class_svm"]] += 0.6
    return int(max(counter, key=counter.get))

  def get_prediction_nose(row):
    counter = {0: 0, 1: 0}
    counter[row["Predicted_class_cnn"]] += 0.42
    counter[row["Predicted_class_lightGBM"]] += 0.1
    counter[row["Predicted_class_alexnet"]] += 0.6
    counter[row["Predicted_class_svm"]] += 0.8
    return int(max(counter, key=counter.get))

  if feature == "eyes":
    return get_prediction_eyes(row)
  if feature == "mouth":
    return get_prediction_mouth(row)
  if feature == "nose":
    return get_prediction_nose(row)

In [ ]:
def get_final_prediction(row, category, feature):
  if category == "age":
    return get_predictions_age(row, feature)
  if category == "ethnicity":
    return get_predictions_ethnicity(row, feature)
  if category == "gender":
    return get_predictions_gender(row, feature)

In [ ]:
def main():
  folder_path = "/content/drive/MyDrive/biometric_systems_project_cropped_images/ensemble_learning"
  category = ["age", "ethnicity", "gender"]
  features = ["eyes", "mouth", "nose"]
  for c in category:
    print(c)
    for feature in features:
      feature_path = os.path.join(folder_path, c, feature)
      merged = merged_results(feature_path)
      merged["Final_prediction"] = merged.apply(lambda row: get_final_prediction(row, c, feature), axis=1)
      print(f"feature: {feature}")
      print(accuracy_score(merged["True_Class"], merged["Final_prediction"]))
      print(classification_report(merged["True_Class"], merged["Final_prediction"], zero_division=0))

if __name__ == "__main__":
  main()

age
feature: eyes
0.6281995661605206
              precision    recall  f1-score   support

           0       0.88      0.90      0.89       399
           1       0.48      0.45      0.46       197
           2       0.42      0.27      0.33       279
           3       0.69      0.85      0.76      2042
           4       0.37      0.30      0.33       794
           5       0.60      0.46      0.52       735
           6       0.52      0.33      0.41       145
           7       0.67      0.11      0.18        19

    accuracy                           0.63      4610
   macro avg       0.58      0.46      0.49      4610
weighted avg       0.61      0.63      0.61      4610

feature: mouth
0.6151843817787419
              precision    recall  f1-score   support

           0       0.79      0.81      0.80       399
           1       0.46      0.39      0.42       197
           2       0.36      0.17      0.23       277
           3       0.68      0.84      0.75      2044
       